In [1]:
import torch
import pandas as pd
import pyaging as pya
import os

In [2]:
os.system("git clone https://github.com/SunScript0/ATAC-clock.git")

Cloning into 'ATAC-clock'...
Updating files: 100% (620/620), done.


0

In [3]:
df = pd.read_table('ATAC-clock/clocks/parallel/2023-02-03_14-38_tpm/final_coefs.tsv', index_col=0)

df = df[df.coef != 0]

df['feature'] = df.index.tolist()
df['coefficient'] = df['coef']

In [4]:
features = df['feature'][0:-1].tolist()

weights = torch.tensor(df['coefficient'][0:-1].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][-1]])

In [5]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=228, out_features=1, bias=True)
)

In [6]:
weights_dict = {
    'preprocessing': 'log1p', 
    'preprocessing_helper': None,
    'postprocessing': None,
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
}

metadata_dict = {
    'species': 'Homo sapiens',
    'data_type': 'atac',
    'year': 2023,
    'citation': "Morandini, Francesco, et al. \"ATAC-clock: An aging clock based on chromatin accessibility.\" GeroScience (2023): 1-18.",
    'doi': "https://doi.org/10.1007/s11357-023-00986-0"
}

In [7]:
torch.save(weights_dict, '../weights/ocampoatac1.pt')
torch.save(metadata_dict, '../metadata/ocampoatac1.pt')

In [8]:
os.system("rm -r ATAC-clock")

0